## Steps to Build a Naive Bayes Classifier

### **Step 1 : Collect Data**
First, we need a labeled dataset where each document (text) is tagged with its correct class label.  
For example, in spam detection, emails would be labeled as either `"spam"` or `"not spam"`.

### **Step 2 : Preprocess the Text**
Before applying Naive Bayes, it's common to preprocess the text:
- Tokenization
- Lowercasing
- Stopword Removal
- Stemming/Lemmatization

### **Step 3 : Calculate Probabilities**
For each class, we need to estimate:  

1. **Prior probability** $P(c)$ — probability of a class occurring in the dataset.  
$$
P(c) = \frac{\text{Number of documents in class } c}{\text{Total number of documents}}
$$

2. **Conditional probabilities** $P(w \mid c)$ — probability of each word $w$ given the class $c$.  
Using the multinomial model with Laplace smoothing:
$$
P(w \mid c) = \frac{\text{Count}(w \text{ in class } c) + 1}{\sum_{w' \in V} \text{Count}(w' \text{ in class } c) + |V|}
$$  
where $V$ is the vocabulary.

### **Step 4 : Apply Bayes Theorem**
For a new document $d$ with words $w_1, w_2, \dots, w_n$, compute the posterior probability for each class $c$:
$$
P(c \mid d) \propto P(c) \prod_{i=1}^{n} P(w_i \mid c)
$$

Then choose the class with the highest posterior probability:
$$
\hat{c} = \arg\max_c P(c) \prod_{i=1}^{n} P(w_i \mid c)
$$

### **Step 5 : Predict the Class**
For a given document, calculate the posterior probability for each class and classify the document as belonging to the class with the highest posterior probability.



## Example

In [1]:
# Count the number of documents in each class
class_count = {"Animal" : 2, "Vehicle":2}

#Calculate prior probabilities
total_documents = sum(class_count.values())
prior_prob = {label: count/total_documents for label, count in class_count.items()}

print("Prior Probabilities:", prior_prob)

Prior Probabilities: {'Animal': 0.5, 'Vehicle': 0.5}


In [5]:
#Define the vocabulary and word counts for each class
vocabulary = ["the", "cat","chases","mouse","dog","barks","loudly","car","is","fast","truck","large"]
word_count = {
    "Animal": {"the":2, "cat":1, "chases":1, "mouse":1, "dog":1, "barks":1, "loudly":1},
    "Vehicle": {"the":2, "car":1, "is":2, "fast":1, "truck":1, "large":1}
}

#Calculate conditional probabilities
total_word_count = {"Animal":7, "Vehicle":7}
cond_prob = {}

for label in ["Animal", "Vehicle"]:
    cond_prob[label]= {}
    total_words = sum(word_count[label].values())

    for word in vocabulary:
        # Laplace smoothing: +1 in numerator, +|V| in denominator
        cond_prob[label][word] = (word_count[label].get(word,0) + 1) / (total_words + len(vocabulary))

print("Conditional Probabilities:", cond_prob)

Conditional Probabilities: {'Animal': {'the': 0.15, 'cat': 0.1, 'chases': 0.1, 'mouse': 0.1, 'dog': 0.1, 'barks': 0.1, 'loudly': 0.1, 'car': 0.05, 'is': 0.05, 'fast': 0.05, 'truck': 0.05, 'large': 0.05}, 'Vehicle': {'the': 0.15, 'cat': 0.05, 'chases': 0.05, 'mouse': 0.05, 'dog': 0.05, 'barks': 0.05, 'loudly': 0.05, 'car': 0.1, 'is': 0.15, 'fast': 0.1, 'truck': 0.1, 'large': 0.1}}


In [9]:
import numpy as np

#Calculate posterior probabilities for a new document

def predict_class(document, prior_prob, cond_prob):
    words = document.split()
    max_prob = -float("inf")
    best_class = None

    for label in ["Animal", "Vehicle"]:
        prob = np.log(prior_prob[label])
        for word in words:
            prob += np.log(cond_prob[label].get(word, 1e-5))
        if prob > max_prob:
            max_prob = prob
            best_class = label
    return best_class


# Step 5: Test the model
test_sentences = [
    "the cat chases the mouse",   # Animal
    "the car is fast",            # Vehicle
    "the dog barks loudly",       # Animal
    "the truck is red"          # Vehicle
]

for sentence in test_sentences:
    predicted = predict_class(sentence, prior_prob, cond_prob)
    print(f"Sentence: '{sentence}' -> Predicted class: {predicted}")

Sentence: 'the cat chases the mouse' -> Predicted class: Animal
Sentence: 'the car is fast' -> Predicted class: Vehicle
Sentence: 'the dog barks loudly' -> Predicted class: Animal
Sentence: 'the truck is red' -> Predicted class: Vehicle
